#### Vector stores and Retriever

The Langchain's vector store and retriever abstractions. These abstractions are desigend to support retrieval of data --  from (vector) databases and other sources -- for integration with LLM workflows. They are important for application that fetch data to be reasoned over as part of model inference, as in the case of retrieval - augmented generation. 

- Documents
- Vector Stores
- Retrievers

In [1]:
!pip install langchain
!pip install langchain-chroma
!pip install langchain-groq

## Documents

Langchain implements a Documents abstraction, which is intended to represent a unit of text and associated metadata. It has tow attributes:

- page_content: A string representing the content;
- metadata: A dict containing arbitrary metadata.

The Metadata attribute can capture information about the source of the document, its relationship ot other documents, and other information. Note that an individual Document object often represents a chunk of larger document.

Generate Some sample documents:

In [2]:
from langchain_core.documents import Document

doc = [
    Document(page_content="Hello, world!", metadata={"source": "greeting.txt"}),
    Document(page_content="LangChain is great for building applications with LLMs.", metadata={"source": "langchain.txt"}),
    Document(page_content="LLMs are powerful tools for natural language processing.", metadata={"source": "llm.txt"}),
    Document(page_content="Python is a versatile programming language.", metadata={"source": "python.txt"}),
    Document(page_content="Chroma is a vector database for storing embeddings.", metadata={"source": "chroma.txt"}),
]


In [3]:
print(doc)

[Document(metadata={'source': 'greeting.txt'}, page_content='Hello, world!'), Document(metadata={'source': 'langchain.txt'}, page_content='LangChain is great for building applications with LLMs.'), Document(metadata={'source': 'llm.txt'}, page_content='LLMs are powerful tools for natural language processing.'), Document(metadata={'source': 'python.txt'}, page_content='Python is a versatile programming language.'), Document(metadata={'source': 'chroma.txt'}, page_content='Chroma is a vector database for storing embeddings.')]


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = ChatGroq(groq_api_key = groq_api_key, model = "llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x00000271F43403A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000271F43402B0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
!pip install langchain_huggingface

In [7]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [9]:
## Vector Store 
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents=doc, embedding=embeddings)
vectorstore

In [11]:
vectorstore.similarity_search("programming")

[Document(id='3d4594d0-3587-4e7d-a3f5-049df7c392c1', metadata={'source': 'python.txt'}, page_content='Python is a versatile programming language.'),
 Document(id='296b3abf-e19b-4327-9d77-6a606861fa40', metadata={'source': 'python.txt'}, page_content='Python is a versatile programming language.'),
 Document(id='9a124622-866b-40ac-ab5e-d09dc21ba46a', metadata={'source': 'llm.txt'}, page_content='LLMs are powerful tools for natural language processing.'),
 Document(id='a7f48f0b-1fd3-467a-82fd-cda58969be2b', metadata={'source': 'llm.txt'}, page_content='LLMs are powerful tools for natural language processing.')]

In [12]:
## Async query
await vectorstore.asimilarity_search("programming")

[Document(id='3d4594d0-3587-4e7d-a3f5-049df7c392c1', metadata={'source': 'python.txt'}, page_content='Python is a versatile programming language.'),
 Document(id='296b3abf-e19b-4327-9d77-6a606861fa40', metadata={'source': 'python.txt'}, page_content='Python is a versatile programming language.'),
 Document(id='9a124622-866b-40ac-ab5e-d09dc21ba46a', metadata={'source': 'llm.txt'}, page_content='LLMs are powerful tools for natural language processing.'),
 Document(id='a7f48f0b-1fd3-467a-82fd-cda58969be2b', metadata={'source': 'llm.txt'}, page_content='LLMs are powerful tools for natural language processing.')]

In [13]:
## Similarity search with score
vectorstore.similarity_search_with_score("programming")

[(Document(id='3d4594d0-3587-4e7d-a3f5-049df7c392c1', metadata={'source': 'python.txt'}, page_content='Python is a versatile programming language.'),
  0.8644969463348389),
 (Document(id='296b3abf-e19b-4327-9d77-6a606861fa40', metadata={'source': 'python.txt'}, page_content='Python is a versatile programming language.'),
  0.8644969463348389),
 (Document(id='9a124622-866b-40ac-ab5e-d09dc21ba46a', metadata={'source': 'llm.txt'}, page_content='LLMs are powerful tools for natural language processing.'),
  1.5134737491607666),
 (Document(id='a7f48f0b-1fd3-467a-82fd-cda58969be2b', metadata={'source': 'llm.txt'}, page_content='LLMs are powerful tools for natural language processing.'),
  1.5134737491607666)]

## Retrievers

LangChain vectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set fo methods. (eg, synchronous and asynchronous invoke and batch operations)  and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselvesm withour subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

## 1st way
retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat", "dog"])

[[Document(id='d4313a14-ffe6-42ea-969a-b531b1bda685', metadata={'source': 'greeting.txt'}, page_content='Hello, world!')],
 [Document(id='d4313a14-ffe6-42ea-969a-b531b1bda685', metadata={'source': 'greeting.txt'}, page_content='Hello, world!')]]

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retriever include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following.

In [ ]:
## 2nd way
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs= {"k": 1}
)

retriever.batch(["cat", "dog"])

[[Document(id='d4313a14-ffe6-42ea-969a-b531b1bda685', metadata={'source': 'greeting.txt'}, page_content='Hello, world!')],
 [Document(id='d4313a14-ffe6-42ea-969a-b531b1bda685', metadata={'source': 'greeting.txt'}, page_content='Hello, world!')]]

In [ ]:
## Integrate Retriever with chain
## RAG Chain - Retrieval Augmented Generation Chain 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

response = rag_chain.invoke("tell me about dogs")
response.content

'There is no information about dogs in the provided context. The context appears to be a reference to a document with a greeting message, but it does not contain any information about dogs.'